## The Most Environmentally Friendly Neighborhood

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas
cans = pd.read_csv('PittsburghSmartTrash.csv')
neighborhood = pd.read_csv('neighborhoods.csv')
fire = pd.read_csv('FireIncidents.csv')

In [7]:
sortedarea = neighborhood.sort_values(by = "hood")
sortedareadf = sortedarea[['hood','acres']]
newsortedareadf = sortedareadf.reset_index(drop=True)
newsortedareadf

,hood,acres
0,Allegheny Center,134.703
1,Allegheny West,90.841
2,Allentown,189.608
3,Arlington,302.320
4,Arlington Heights,84.592
...,...,...
85,Upper Lawrenceville,260.349
86,West End,128.679
87,West Oakland,138.372
88,Westwood,445.009


In [8]:
# this creates a new table to show the ratio of fires to acres of neighborhood
new_df = fire.sort_values(["neighborhood"], ascending=True)
fire['neighborhood'].value_counts()

newdf =fire['neighborhood'].value_counts().rename_axis('neighborhood').reset_index(name='frequency')
sorteddf = newdf.sort_values(["neighborhood"], ascending=True)
sortednew = sorteddf.reset_index(drop=True)
mergesorteddata = sortednew.merge(newsortedareadf, left_on = 'neighborhood', right_on = 'hood', suffixes=('frequency', 'acres'))
mergesorteddata.sort_values(['frequency'], ascending=True)
mergesorteddata['ratio'] = mergesorteddata['acres']/mergesorteddata['frequency'] 
ratio = mergesorteddata.sort_values(["ratio"], ascending=False)
r = ratio.drop(columns='hood')
r.head(20)

,neighborhood,frequency,acres,ratio
37,Hays,18,1125.478,62.526556
26,East Carnegie,10,277.895,27.789500
31,Fairywood,23,622.746,27.075913
54,New Homestead,20,510.728,25.536400
81,Swisshelm Park,15,308.575,20.571667
66,Ridgemont,12,237.059,19.754917
88,Westwood,23,445.009,19.348217
77,St. Clair,11,199.285,18.116818
65,Regent Square,7,123.682,17.668857
89,Windgap,17,266.678,15.686941


In [26]:
#created a new table merging the two together and calculating the ratio
# this table refers to the smart trash cans and the ratio of that to acreage of neighborhood
new_df= cans.sort_values(["neighborhood"], ascending=False)

sort = neighborhood.sort_values(by = "hood")
sorted = sort[['hood','acres']]
new = sorted.reset_index(drop=True)

n =cans['neighborhood'].value_counts().rename_axis('neighborhood').reset_index(name='frequency')
df = n.sort_values(["neighborhood"], ascending=True)
s = df.reset_index(drop=True)
merge = s.merge(new, left_on = 'neighborhood', right_on = 'hood', suffixes=('frequency', 'acres'))
merge['ratio'] = merge['frequency']/merge['acres']
finalresult = merge.sort_values(["ratio"], ascending=False)
finalresult= finalresult.drop(columns='hood')
true = finalresult.drop(labels=51) ## to take out Perry North since it is 0 acres, causing an infinite number
true

,neighborhood,frequency,acres,ratio
48,North Shore,52,195.014,0.266648
17,Central Oakland,39,180.883,0.215609
25,East Liberty,68,373.650,0.181988
28,Friendship,12,68.297,0.175703
22,East Allegheny,26,151.705,0.171385
...,...,...,...,...
68,Stanton Heights,2,472.615,0.004232
78,Windgap,1,266.678,0.003750
23,East Carnegie,1,277.895,0.003598
70,Swisshelm Park,1,308.575,0.003241


In [10]:
# import pandas as pd
df = pd.read_csv('Pittsburgh_Beautify_The_Burgh.csv')
dataSetTwo = df[['Neighborhood','SHAPE_Area']]
#dataSetTwo = df.groupby('Neighborhood').mean().sort_values(['SHAPE_Area','Neighborhood'],ascending =[False,True])
#dataSetTwo['SHAPE_Area'] = dataSetTwo['SHAPE_Area']/4046.86
sortDataSetTwo = dataSetTwo.sort_values(by="Neighborhood")
df2 = sortDataSetTwo.groupby(['Neighborhood']).sum()
#mergedf2 = sortDataSetTwo.groupby(by=sortDataSetTwo.columns, axis=1).sum
#mergedf2 = sortDataSetTwo.groupby(['Neighborhood']).agg({'SHAPE_Area' : 'sum'})
#mergedf2
df2['SHAPE_Area'] = df2['SHAPE_Area']/4046.8564224
#df2
df2['Neighborhood'] = df2.index
df2.reset_index(drop=True)
columns_titles = ["Neighborhood","SHAPE_Area"]
df2=df2.reindex(columns=columns_titles)
df2=df2['SHAPE_Area'].rename_axis('Neighborhood').reset_index(name='area')
df2
##newdf =fire_incident['neighborhood'].value_counts().rename_axis('neighborhood').reset_index(name='frequency')

,Neighborhood,area
0,Allegheny West,4.732557
1,Allentown,3.749644
2,Arlington,5.947094
3,Beltzhoover,23.596112
4,Bluff/Uptown,2.270216
5,Carrick,3.781603
6,Central Business District,9.649654
7,Central Oakland,50.399079
8,East Liberty,7.738735
9,Elliott,8.368664


In [11]:
df1 = neighborhood.sort_values(by = "hood")
dfaa = df1[['hood','acres']]
df3 = dfaa.reset_index(drop=True)

newdf =df['Neighborhood'].value_counts().rename_axis('Neighborhood').reset_index(name='area')
sorteddf = newdf.sort_values(["Neighborhood"], ascending=True)
sortednew = sorteddf.reset_index(drop=True)
mergesorteddata = sortednew.merge(df3, how = 'left', left_on = 'Neighborhood', right_on = 'hood', suffixes=('area', 'acres'))
mergesorteddata.sort_values(['area'], ascending=True)
mergesorteddata['ratio'] = mergesorteddata['area'] / mergesorteddata['acres']
ratio = mergesorteddata.sort_values(["ratio"], ascending=True)
addcol = df2['area']
ratio.insert(1, 'shapearea', addcol)
ratio['realratio'] = ratio['shapearea'] / ratio['acres']
del ratio["area"]
del ratio["hood"]
del ratio["ratio"]
ratio
ratio = ratio.sort_values(["realratio"], ascending=True)
ratio = ratio.dropna()
ratio

,Neighborhood,shapearea,acres,realratio
11,Hazelwood,2.810837,1017.805,0.002762
5,Carrick,3.781603,1075.670,0.003516
19,Sheraden,3.959767,572.125,0.006921
10,Greenfield,3.894347,497.028,0.007835
18,Shadyside,6.987445,592.104,0.011801
17,Point Breeze,10.301870,645.695,0.015955
12,Homewood South,4.766865,256.093,0.018614
2,Arlington,5.947094,302.320,0.019672
1,Allentown,3.749644,189.608,0.019776
8,East Liberty,7.738735,373.650,0.020711


In [147]:
# combining both the ratio from the fire incidents and the ratio from the smart trash cans
rt = r.merge(true, how='left', left_on='neighborhood', right_on='neighborhood',suffixes=('cans','fire'))
rt['ratiofire'] = rt['ratiofire'].fillna(0).astype(int)
rt["ratio"] = rt["ratiofire"] + rt["ratiocans"]
rt

,neighborhood,frequencycans,acrescans,ratiocans,frequencyfire,acresfire,ratiofire,ratio
0,Hays,18,1125.478,62.526556,NaN,NaN,0,62.526556
1,East Carnegie,10,277.895,27.789500,1.0,277.895,0,27.789500
2,Fairywood,23,622.746,27.075913,NaN,NaN,0,27.075913
3,New Homestead,20,510.728,25.536400,NaN,NaN,0,25.536400
4,Swisshelm Park,15,308.575,20.571667,1.0,308.575,0,20.571667
...,...,...,...,...,...,...,...,...
85,East Allegheny,111,151.705,1.366712,26.0,151.705,0,1.366712
86,Knoxville,147,193.982,1.319605,6.0,193.982,0,1.319605
87,Marshall-Shadeland,119,148.209,1.245454,15.0,148.209,0,1.245454
88,Lincoln-Lemington-Belmar,174,180.105,1.035086,6.0,180.105,0,1.035086


In [148]:
y = rt.merge(ratio, how = "left", left_on = "neighborhood", right_on ="Neighborhood", suffixes=("ratio","realratio"))
y['realratio'] = y['realratio'].fillna(0)
y['realratio'] = y['realratio'].astype(int)
y["realratio"] = y["realratio"]
y["totalratio"] = y["ratio"] + y["realratio"]
y.head(30)

,neighborhood,frequencycans,acrescans,ratiocans,frequencyfire,acresfire,ratiofire,ratio,Neighborhood,shapearea,acres,realratio,totalratio
0,Hays,18,1125.478,62.526556,NaN,NaN,0,62.526556,NaN,NaN,NaN,0,62.526556
1,East Carnegie,10,277.895,27.789500,1.0,277.895,0,27.789500,NaN,NaN,NaN,0,27.789500
2,Fairywood,23,622.746,27.075913,NaN,NaN,0,27.075913,NaN,NaN,NaN,0,27.075913
3,New Homestead,20,510.728,25.536400,NaN,NaN,0,25.536400,NaN,NaN,NaN,0,25.536400
4,Swisshelm Park,15,308.575,20.571667,1.0,308.575,0,20.571667,NaN,NaN,NaN,0,20.571667
5,Ridgemont,12,237.059,19.754917,NaN,NaN,0,19.754917,NaN,NaN,NaN,0,19.754917
6,Westwood,23,445.009,19.348217,5.0,445.009,0,19.348217,NaN,NaN,NaN,0,19.348217
7,St. Clair,11,199.285,18.116818,1.0,199.285,0,18.116818,NaN,NaN,NaN,0,18.116818
8,Regent Square,7,123.682,17.668857,9.0,123.682,0,17.668857,NaN,NaN,NaN,0,17.668857
9,Windgap,17,266.678,15.686941,1.0,266.678,0,15.686941,NaN,NaN,NaN,0,15.686941


In [123]:
c = y[["neighborhood", "totalratio"]]
c

,neighborhood,totalratio
0,Hays,62.526556
1,East Carnegie,27.789500
2,Fairywood,27.075913
3,New Homestead,25.536400
4,Swisshelm Park,20.571667
...,...,...
85,East Allegheny,1.366712
86,Knoxville,1.319605
87,Marshall-Shadeland,1.245454
88,Lincoln-Lemington-Belmar,1.035086


In [131]:
neighborhoods = geopandas.read_file("Neighborhoods/Neighborhoods_.shp") # read in the shapefile
map = neighborhoods.merge(c, how='left', left_on='hood', right_on='neighborhood')
# look at the head to confirm it merged correctly
map[['neighborhood','totalratio']].head()

,neighborhood,totalratio
0,Central Oakland,2.179313
1,North Oakland,2.837867
2,West Oakland,2.515855
3,Crawford-Roberts,1.866303
4,Strip District,8.316255


In [144]:
map.plot(column='totalratio', # set the data to be used for coloring
               cmap='Greens',              # choose a color palette
               edgecolor="white",        # outline the districts in white
               legend=True,              # show the legend
               legend_kwds={'label': "Environmental friendliness"}, # label the legend
               figsize=(15, 10),         # set the size
               missing_kwds={"color": "lightgrey"} # set disctricts with no data to gray
               )

/opt/tljh/user/lib/python3.8/site-packages/geopandas/plotting.py:373: UserWarning: The GeoSeries you are attempting to plot is empty. Nothing has been displayed.
  warnings.warn(


ValueError: cannot convert float NaN to integer

ValueError: cannot convert float NaN to integer

Conclusion Paragraphs:
<br/>
Ruchi: 
According to my personal notebook, the neighborhood with the largest ratio of trash cans to area is North Shore and the one with the least is Banksville. That is different from what the total ratio in the combined notebook tells us, however. That is because of the difference in the metric of my personal notebook and the combined noteboook. Overall, the metric of my personal one was only based off of one dataset so it did not take into consideration the other two datasets that we used in the combined. My personal metric was the number of trash cans per acre to get which neighborhood had the most trash cans and then that was added with the other two ratios to get a total ratio which weighed each metric equally. 

Dailynn:
According to my notebook, the neighborhood with the largest area cleaned to area ratio is Mount Washington and the least is Hazelwood. That is different to the other notebooks because of the metrics. My metric was using the dataset to find out which neighborhood was the cleanest by doing the area of the cleaning and dividing it by the total area of the neighborhood. Through that I was able to create the ratio and compare the neighborhoods.

Le Lin: 
According to the ratio of my personal data, the best neighborhood is Hays and the worst is Lincoln-Lemington-Belmar. My metric is the acre/frequency, so if the ratio is higher/lower than there are less/more fire incidents.

All three of our datasets required the use of the Neighborhoods dataset because depending on the area of the neighborhood it could have more or less fires, smart trash cans, and area cleaned. 